## Bot or Not Bot

In [1]:
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import *
import matplotlib.pyplot as plt
import utils
import numpy as np
import os
import importlib
from xgboost import XGBClassifier
import pickle

In [2]:
pd.options.display.max_columns = 999

### Read in Data

In [3]:
cresci_dir_2017 = '../data/cresci_2017/'
cresci_dir_2015 = '../data/cresci_2015/'
varol = '../data/varol_users/'

In [4]:
real_users_1 = pd.read_csv(os.path.join(cresci_dir_2017, 'genuine_accounts.csv/users.csv'), index_col=0)
real_users_2 = pd.read_csv(os.path.join(cresci_dir_2015, 'E13/users.csv'), index_col=0)
real_users_3 = pd.read_csv(os.path.join(cresci_dir_2015, 'TFP/users.csv'), index_col=0)
real_users_4 = pd.read_csv(os.path.join(varol, 'varol_real.csv'), index_col=0)

# real_tweets = pd.read_csv(os.path.join(cresci_dir_2017, 'genuine_accounts.csv/tweets.csv'), index_col=0, low_memory=False)
spam_users_1 = pd.read_csv(os.path.join(cresci_dir_2017, 'social_spambots_1.csv/users.csv'), index_col=0)
# spam_tweets_1 = pd.read_csv(os.path.join(cresci_dir_2017, 'social_spambots_1.csv/tweets.csv'), index_col=0, low_memory=False)
spam_users_2 = pd.read_csv(os.path.join(cresci_dir_2017, 'social_spambots_2.csv/users.csv'), index_col=0)
# spam_tweets_2 = pd.read_csv(os.path.join(cresci_dir_2017, 'social_spambots_2.csv/tweets.csv'), index_col=0, low_memory=False)
spam_users_3 = pd.read_csv(os.path.join(cresci_dir_2017, 'social_spambots_3.csv/users.csv'), index_col=0)
# spam_tweets_3 = pd.read_csv(os.path.join(cresci_dir_2017, 'social_spambots_3.csv/tweets.csv'), index_col=0, low_memory=False)
spam_users_4 = pd.read_csv(os.path.join(cresci_dir_2017, 'traditional_spambots_1.csv/users.csv'), index_col=0)
# spam_tweets_4 = pd.read_csv(os.path.join(cresci_dir_2017, 'traditional_spambots_1.csv/tweets.csv'), index_col=0, low_memory=False)
spam_users_5 = pd.read_csv(os.path.join(cresci_dir_2017, 'traditional_spambots_2.csv/users.csv'), index_col=0)
spam_users_6 = pd.read_csv(os.path.join(cresci_dir_2017, 'traditional_spambots_3.csv/users.csv'), index_col=0)
spam_users_7 = pd.read_csv(os.path.join(cresci_dir_2017, 'traditional_spambots_4.csv/users.csv'), index_col=0)
spam_users_8 = pd.read_csv(os.path.join(cresci_dir_2015, 'FSF/users.csv'), index_col=0)
spam_users_9 = pd.read_csv(os.path.join(cresci_dir_2015, 'INT/users.csv'), index_col=0)
spam_users_10 = pd.read_csv(os.path.join(cresci_dir_2015, 'TWT/users.csv'), index_col=0)
spam_users_11 = pd.read_csv(os.path.join(varol, 'varol_fake.csv'), index_col=0)

In [5]:
# Leave spam_user_3 for test
spam_users_train = pd.concat(objs=[spam_users_1, spam_users_2, spam_users_3, spam_users_4, spam_users_5, spam_users_7,
                                  spam_users_8, spam_users_9, spam_users_10, spam_users_11, spam_users_6])
# spam_tweets_train = pd.concat(objs=[spam_tweets_2])
spam_users_test = spam_users_6
# spam_tweets_test = spam_tweets_1

# Split geniune user data set to train and test
# real_users_train = real_users.loc[real_users_1['test_set_1']==0, :]
# real_users_test = real_users.loc[real_users_1['test_set_1']==1, :]

# Concatenate cresci-2015 real users data
real_users_train = pd.concat(objs=[real_users_1, real_users_3, real_users_2, real_users_4])
real_users_test = pd.concat(objs=[real_users_4])

### Add label

In [6]:
real_users_train.loc[:,'label'] = np.zeros(shape=(real_users_train.shape[0])) 
real_users_test.loc[:,'label'] = np.zeros(shape=(real_users_test.shape[0]))
spam_users_test.loc[:,'label'] = np.ones(shape=(spam_users_test.shape[0]))
spam_users_train.loc[:,'label'] = np.ones(shape=(spam_users_train.shape[0]))

# concat the genuine users data with the spam data
train_set = pd.concat([real_users_train, spam_users_train])
test_set = pd.concat([real_users_test, spam_users_test])

### Feature engineering

In [8]:
# tweet_feats = utils.extract_tweet_feats(all_tweets)

# # Merge features from tweets to users
# all_users = all_users.merge(tweet_feats, how='inner', left_index=True, right_index=True)

reload(utils)
# ## Feature selection
feat_choices = ['default_profile', 'default_profile_image', 'favourites_count', 'followers_count',
               'friends_count', 'geo_enabled', 'listed_count', 'profile_use_background_image',
               'statuses_count', 'verified', 'protected', 'profile_background_tile', 'profile_background_color',
               'profile_sidebar_border_color', 'profile_sidebar_fill_color']

# Training set
features, labels, scaler, feat_names = utils.feat_processing(train_set, feat_choices)
# Test set 
test_features, test_labels = utils.feat_processing(test_set, feat_choices, scaler)

In [20]:
pickle.dump(scaler, open('standardscalar.dat', 'wb'))

### Test with my own twitter account just for fun

In [9]:
# temp = pd.read_csv('stacia.csv', index_col=0)
# temp['label'] = 0
# test_features, test_labels = utils.feat_processing(temp, feat_choices, scaler)

### Split dataset to three

In [10]:
# all_data = pd.concat(objs=[train_set, test_set])
# features = all_data.loc[:, all_data.columns != 'labels']
# labels = all_data.loc[:,'label']
# x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=392)
# x_eval, x_test, y_eval, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=392)
# features, labels, scaler, feat_names = utils.feat_processing(x_train, feat_choices)
# # Test set 
# test_features, test_labels = utils.feat_processing(x_test, feat_choices, scaler)

### Train a gradient boosting classifier 

In [11]:
# # gb_classifier = GradientBoostingClassifier()
gb_classifier = XGBClassifier()
gb_classifier.fit(features, labels)
prediction = gb_classifier.predict(test_features)

In [14]:
pickle.dump(gb_classifier, open('bot_classifer.dat', 'wb'))

In [16]:
test = pickle.load(open('bot_classifer.dat', 'rb'))

In [12]:
idx = np.argsort(gb_classifier.feature_importances_)[::-1]
print 'Importance of each feature:\n'
feat_order = [str(y) + '. ' + str(feat_names[x]) for x,y in zip(idx, range(1, len(feat_names)+1))]

for feat in feat_order:
    print feat

print '\n'
print 'Classification Report:\n'
print classification_report(test_labels, prediction)

Importance of each feature:

1. friends_count
2. favourites_count
3. followers_count
4. statuses_count
5. listed_count
6. profile_background_color
7. geo_enabled
8. profile_use_background_image
9. default_profile
10. profile_background_tile
11. default_profile_image
12. real_sidebar_color
13. spam_sidebar_color
14. profile_sidebar_fill_color
15. protected
16. verified


Classification Report:

             precision    recall  f1-score   support

        0.0       0.98      0.95      0.96       681
        1.0       0.92      0.96      0.94       403

avg / total       0.96      0.95      0.95      1084



In [13]:
confusion_matrix(test_labels, prediction)

array([[647,  34],
       [ 15, 388]])

### Other code

In [502]:
# Grid search cv code
# params = {'max_depth':[2, 3, 5], 'min_samples_leaf':[1,2,3,5], 
#           'min_samples_split':[2,3,5], 'n_estimators':[50,100,200,500]}
# clf = GridSearchCV(gb_classifier, params)
# clf.fit(x_train, y_train)

# Features include tweets
# feat_choices = ['default_profile', 'default_profile_image', 'favourites_count', 'followers_count',
#                'friends_count', 'geo_enabled', 'listed_count', 'profile_use_background_image',
#                'statuses_count', 'verified', 'tweet_len', 'contain_RT', 'num_mentions', 'num_hashtags', 
#                'num_urls', 'retweet_count']
